### Import Library

In [1]:
import pandas as pd
import numpy as np
import textwrap
import datetime

# inisialisasi now sebagai nama file 
now = datetime.datetime.today().strftime('%d_%m_%Y')

### Proses Data Dosen

In [2]:
dosen_2023 = pd.read_excel(f'data//2023.xlsx')
dosen_2023 = dosen_2023[['NIP', 'NAMA (BERSERA GELAR)', 'JK', 'PEND', 'GOL', 'JABFUNG', 'STATUS PEGAWAI',  
                        'FAKULTAS']]
dosen_2023 = dosen_2023.rename(columns = {'NIP': 'nip', 'NAMA (BERSERA GELAR)': 'nama', 'STATUS PEGAWAI': 'status_pegawai',
                                          'JK': 'gender', 'PEND': 'pend_terakhir', 'GOL': 'gol', 'JABFUNG': 'jabfung',
                                          'FAKULTAS': 'fakultas'})

# fungsi untuk ubah gender menjadi Laki-laki dan Perempuan
def change_gender(col):
    if col == 'L':
        col = 'Laki-laki'
    elif col == 'P':
        col = 'Perempuan'
    return col

# fungsi untuk replace beberapa data nip yang tidak sesuai format
def replace_data_2023(df):
    df['nip'].iat[1463] = '020150819560721101'
    df['nip'].iat[1464] = '020150819590722201'
    df['nip'].iat[1465] = '020190319571017101'
    df['nip'].iat[1466] = '020190319650315201'
    
    return df

# fungsi untuk extract usia dosen
def extract_usia(df):
    
    # mark sebagai penanda penghitung usia
    mark = 0
    batas = 1109
    mark = 1
        
    # proses data bagian pertama
    df1 = df[:batas].copy()
    df1['nip'] = df1['nip'].astype(str)
    df1['usia'] = df1['nip'].str.slice(0, -10)
    df1['usia'] = pd.to_datetime(df1['usia'], format='%Y%m%d')
    today = pd.Timestamp.now()
    
    # kondisi jika mark == 1 maka tahun now diset sebagai tahun sekarang
    if mark == 1:
        now = pd.Timestamp('now')
    # kondisi jika mark != 1 maka tahun pengurang disesuaikan dengan counter
    else:
        now = today - pd.Timedelta(days=365 * counter)
    # extract usia
    df1['usia'] = (now.year - df1['usia'].dt.year) - ((now.month - df1['usia'].dt.month) < 0)
    
    # proses data bagian kedua
    df2 = df[batas:].copy()
    df2['nip'] = df2['nip'].astype(str)
    df2['usia'] = df2['nip'].str.slice(7, -3)
    df2['usia'] = pd.to_datetime(df2['usia'], format='%Y%m%d')
    df2['usia'] = (now.year - df2['usia'].dt.year) - ((now.month - df2['usia'].dt.month) < 0)

    # sambung data bagian pertama dan kedua menjadi satu 
    df = pd.concat([df1, df2], ignore_index=True)
    return df

def extract_substr(text, mode):
    string = text.split(',', 1)

    string = string[0]    
    string = string.replace('Dipl. ', '')
    string = string.replace('Dr. ', '')
    string = string.replace('Eng. ', '')
    string = string.replace('dr. ', '')
    string = string.replace('phil. ', '')
    string = string.replace('Dra. ', '')
    string = string.replace('Drs. ', '')
    string = string.replace('H. ', '')
    string = string.replace('Hj. ', '')
    string = string.replace('Ir. ', '')
    string = string.replace('Prof. ', '')
    string = string.replace('paed. ', '')
    string = string.replace('Phil. ', '')
    
    string = string.replace('M.A.', '')
    string = string.replace('M.ED.', '')
    string = string.replace('PH.D', '')
    string = string.replace('M.SI', '')
    string = string.replace('M.PD', '')
    string = string.replace('S.PD', '')
    string = string.replace('S.PSI', '')
    string = string.replace('PSIKOLOG', '')
    string = string.replace('S.E', '')
    string = string.replace('M.S.M', '')
    string = string.replace('M.DS.', '')
    string = string.replace('DR. ', '')
    string = string.replace('DR.', '')
    string = string.strip()
    
    return string

# apply fungsi pada kolom nama kemudian buat kolom baru 'Nama Tanpa Gelar'
dosen_2023['Nama Tanpa Gelar'] = dosen_2023['nama'].apply(lambda x: extract_substr(x, 0))
dosen_2023['Nama Tanpa Gelar'] = dosen_2023['Nama Tanpa Gelar'].apply(lambda x: x.upper())

# apply fungsi untuk menghasilkan format data yang sesuai
dosen_2023['gender'] = dosen_2023['gender'].apply(lambda x: change_gender(x))
dosen_2023 = replace_data_2023(dosen_2023)
dosen_2023 = extract_usia(dosen_2023)
dosen_2023.head()

,nip,nama,gender,pend_terakhir,gol,jabfung,status_pegawai,fakultas,Nama Tanpa Gelar,usia
0,196202081986011002,"Prof. Dr. H. M. Solehuddin, M.Pd., M.A.",Laki-laki,S3,IV/c,Guru Besar,PNS,FIP,M. SOLEHUDDIN,61
1,195306031980021003,"Prof. Dr. H. Mohammad Ali, M.Pd., M.A.",Laki-laki,S3,IV/e,Guru Besar,PNS,FIP,MOHAMMAD ALI,69
2,195404021980112001,"Prof. Dr. Hj. Ihat Hatimah, M.Pd.",Perempuan,S3,IV/e,Guru Besar,PNS,FIP,IHAT HATIMAH,68
3,195501011981011001,"Prof. Dr. H. Achmad Hufad, M.Ed.",Laki-laki,S3,IV/e,Guru Besar,PNS,FIP,ACHMAD HUFAD,68
4,196111091987031001,"Prof. Dr. H. Mustofa Kamil, M.Pd.",Laki-laki,S3,IV/e,Guru Besar,PNS,FIP,MUSTOFA KAMIL,61


In [3]:
dosen_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1566 entries, 0 to 1565
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   nip               1566 non-null   object
 1   nama              1566 non-null   object
 2   gender            1566 non-null   object
 3   pend_terakhir     1566 non-null   object
 4   gol               1549 non-null   object
 5   jabfung           1566 non-null   object
 6   status_pegawai    1566 non-null   object
 7   fakultas          1566 non-null   object
 8   Nama Tanpa Gelar  1566 non-null   object
 9   usia              1566 non-null   int64 
dtypes: int64(1), object(9)
memory usage: 122.5+ KB


### Ambil SINTA ID dari Mapping NIP

In [4]:
# read data dosen_mapping
df_map = pd.read_csv('data//dosen_mapping_26_1_2023.csv', encoding='utf-8')
df_map = df_map[['NIP', 'sinta_id']]

# merge kemudian ambil kolom, ubah tipe data
dosen_2023['nip'] = dosen_2023['nip'].astype('Int64')
df_dosen = (pd.merge(dosen_2023, df_map, left_on='nip', right_on='NIP', how='left'))
df_dosen = df_dosen[['nip', 'sinta_id', 'nama', 'gender', 'pend_terakhir', 'gol', 'jabfung', 'status_pegawai', 'fakultas',
                    'Nama Tanpa Gelar']]
df_dosen['sinta_id'] = df_dosen['sinta_id'].astype('Int64')

# df_final.to_csv(f'TEST{now}.csv', index=False)
df_dosen.head()

,nip,sinta_id,nama,gender,pend_terakhir,gol,jabfung,status_pegawai,fakultas,Nama Tanpa Gelar
0,196202081986011002,5995348,"Prof. Dr. H. M. Solehuddin, M.Pd., M.A.",Laki-laki,S3,IV/c,Guru Besar,PNS,FIP,M. SOLEHUDDIN
1,195306031980021003,5980905,"Prof. Dr. H. Mohammad Ali, M.Pd., M.A.",Laki-laki,S3,IV/e,Guru Besar,PNS,FIP,MOHAMMAD ALI
2,195404021980112001,5979054,"Prof. Dr. Hj. Ihat Hatimah, M.Pd.",Perempuan,S3,IV/e,Guru Besar,PNS,FIP,IHAT HATIMAH
3,195501011981011001,5977826,"Prof. Dr. H. Achmad Hufad, M.Ed.",Laki-laki,S3,IV/e,Guru Besar,PNS,FIP,ACHMAD HUFAD
4,196111091987031001,5994180,"Prof. Dr. H. Mustofa Kamil, M.Pd.",Laki-laki,S3,IV/e,Guru Besar,PNS,FIP,MUSTOFA KAMIL


In [5]:
df_dosen.isnull().sum()

nip                   0
sinta_id            169
nama                  0
gender                0
pend_terakhir         0
gol                  17
jabfung               0
status_pegawai        0
fakultas              0
Nama Tanpa Gelar      0
dtype: int64

### Ambil SINTA ID dari Mapping Nama

In [6]:
# read data, ambil beberapa kolom, extract kolom 'Nama Tanpa Gelar'
df_sinta = pd.read_csv('data//dosen_sinta_11-03-2023.csv', encoding='utf-8')
df_sinta = df_sinta[['nama', 'sinta_id']]
df_sinta['Nama Tanpa Gelar'] = df_sinta['nama'].apply(lambda x: extract_substr(x, 0))
df_sinta = df_sinta[['sinta_id', 'Nama Tanpa Gelar']]

# df_sinta.to_csv(f'SINTA{now}.csv', index=False)


# merge data, fill null value pada kolom sinta_id tabel kiri dengan data sinta_id tabel kanan
df_final = df_dosen.merge(df_sinta, on='Nama Tanpa Gelar', how='left')
df_final['sinta_id_x'].fillna(df_final['sinta_id_y'], inplace=True)
df_final.drop('sinta_id_y', axis=1, inplace=True)
df_final.rename(columns={'sinta_id_x': 'sinta_id', 'Nama Tanpa Gelar': 'nama_tanpa_gelar'}, inplace=True)


# hapus duplikasi data dan reset index
df_final = df_final.drop_duplicates().reset_index(drop=True)
df_final = df_final.drop(995)
df_final = df_final.drop(1419)
df_final = df_final.reset_index(drop=True)


df_final['sinta_id'].fillna('0', inplace=True)
# df_final = df_final.dropna(subset=['sinta_id'])
# df_final.to_csv(f'dosen_{now}.csv', index=False)
df_final.head()

,nip,sinta_id,nama,gender,pend_terakhir,gol,jabfung,status_pegawai,fakultas,nama_tanpa_gelar
0,196202081986011002,5995348,"Prof. Dr. H. M. Solehuddin, M.Pd., M.A.",Laki-laki,S3,IV/c,Guru Besar,PNS,FIP,M. SOLEHUDDIN
1,195306031980021003,5980905,"Prof. Dr. H. Mohammad Ali, M.Pd., M.A.",Laki-laki,S3,IV/e,Guru Besar,PNS,FIP,MOHAMMAD ALI
2,195404021980112001,5979054,"Prof. Dr. Hj. Ihat Hatimah, M.Pd.",Perempuan,S3,IV/e,Guru Besar,PNS,FIP,IHAT HATIMAH
3,195501011981011001,5977826,"Prof. Dr. H. Achmad Hufad, M.Ed.",Laki-laki,S3,IV/e,Guru Besar,PNS,FIP,ACHMAD HUFAD
4,196111091987031001,5994180,"Prof. Dr. H. Mustofa Kamil, M.Pd.",Laki-laki,S3,IV/e,Guru Besar,PNS,FIP,MUSTOFA KAMIL


In [7]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1565 entries, 0 to 1564
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   nip               1565 non-null   Int64 
 1   sinta_id          1565 non-null   Int64 
 2   nama              1565 non-null   object
 3   gender            1565 non-null   object
 4   pend_terakhir     1565 non-null   object
 5   gol               1548 non-null   object
 6   jabfung           1565 non-null   object
 7   status_pegawai    1565 non-null   object
 8   fakultas          1565 non-null   object
 9   nama_tanpa_gelar  1565 non-null   object
dtypes: Int64(2), object(8)
memory usage: 125.4+ KB


In [8]:
df_final.isnull().sum()

nip                  0
sinta_id             0
nama                 0
gender               0
pend_terakhir        0
gol                 17
jabfung              0
status_pegawai       0
fakultas             0
nama_tanpa_gelar     0
dtype: int64

In [9]:
duplicate_rows = df_final[df_final['nama'].duplicated(keep=False)]
num_duplicate = len(duplicate_rows)
print('Jumlah duplikasi:', num_duplicate)
print('Baris duplikat:')
print(duplicate_rows)

Jumlah duplikasi: 0
Baris duplikat:
Empty DataFrame
Columns: [nip, sinta_id, nama, gender, pend_terakhir, gol, jabfung, status_pegawai, fakultas, nama_tanpa_gelar]
Index: []


### Gabung Data Skor SINTA dan H Indeks

In [10]:
dosen_sinta = pd.read_csv('data//dosen_sinta_11-03-2023.csv', encoding='utf-8')
dosen_sinta = dosen_sinta[['sinta_id', 'sinta_score', 'sinta_3yr_score', 'scopus_h_index', 'gs_h_index']] 

df_final = (pd.merge(df_final, dosen_sinta, on='sinta_id', how='left'))
df_final = df_final.drop_duplicates().reset_index(drop=True)


df_final['sinta_score'] = df_final['sinta_score'].astype('Int64')
df_final['sinta_3yr_score'] = df_final['sinta_3yr_score'].astype('Int64')
df_final['scopus_h_index'] = df_final['scopus_h_index'].astype('Int64')
df_final['gs_h_index'] = df_final['gs_h_index'].astype('Int64')

# df_final.to_csv(f'data//dosen_{now}.csv', index=False)
df_final.head()

,nip,sinta_id,nama,gender,pend_terakhir,gol,jabfung,status_pegawai,fakultas,nama_tanpa_gelar,sinta_score,sinta_3yr_score,scopus_h_index,gs_h_index
0,196202081986011002,5995348,"Prof. Dr. H. M. Solehuddin, M.Pd., M.A.",Laki-laki,S3,IV/c,Guru Besar,PNS,FIP,M. SOLEHUDDIN,649,248,3,9
1,195306031980021003,5980905,"Prof. Dr. H. Mohammad Ali, M.Pd., M.A.",Laki-laki,S3,IV/e,Guru Besar,PNS,FIP,MOHAMMAD ALI,1039,422,3,28
2,195404021980112001,5979054,"Prof. Dr. Hj. Ihat Hatimah, M.Pd.",Perempuan,S3,IV/e,Guru Besar,PNS,FIP,IHAT HATIMAH,469,207,1,12
3,195501011981011001,5977826,"Prof. Dr. H. Achmad Hufad, M.Ed.",Laki-laki,S3,IV/e,Guru Besar,PNS,FIP,ACHMAD HUFAD,1900,819,6,11
4,196111091987031001,5994180,"Prof. Dr. H. Mustofa Kamil, M.Pd.",Laki-laki,S3,IV/e,Guru Besar,PNS,FIP,MUSTOFA KAMIL,678,249,2,14
